In [1]:
import random, sys, gc, warnings, math, os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from time import time
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.layers import Input, Dense, Concatenate, Dropout
from tensorflow.keras.utils import Sequence

2024-04-28 17:14:03.437514: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-28 17:14:03.440280: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-28 17:14:03.468386: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-28 17:14:05.260463: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
print("Imports are working correctly")

Imports are working correctly


In [3]:
t0 = time()
np.random.seed(13)
random.seed(13)
min_std = 1e-8

In [4]:
class CFG:
    data_path = './../data/'

In [5]:
# load train data
df = pd.read_csv(os.path.join(CFG.data_path, "train.csv"), nrows=1000000)
x = df.iloc[:, 1:557].to_numpy().astype(np.float32)
y = df.iloc[:, 557:].to_numpy().astype(np.float32)

FileNotFoundError: [Errno 2] No such file or directory: './../data/train.csv'

In [ ]:
# read test
df = pd.read_csv(os.path.join(CFG.data_path, "test.csv"))
xt = df.iloc[:, 1:557].to_numpy().astype(np.float32)
del df
gc.collect()

In [ ]:
# norm X
mx = x.mean(axis=0)
sx = np.maximum(x.std(axis=0), min_std)
x = (x - mx.reshape(1,-1)) / sx.reshape(1,-1)
xt = (xt - mx.reshape(1,-1)) / sx.reshape(1,-1)

In [ ]:
# norm Y
my = y.mean(axis=0)
sy = np.maximum(np.sqrt((y*y).mean(axis=0)), min_std)
y = (y - my.reshape(1,-1)) / sy.reshape(1,-1)

In [ ]:
# data generator
class data_gen(Sequence):  # constructor: save all data locally
    def __init__(self, i1, i2, batch_size):
        self.i1, self.i2 = i1, i2
        self.batch_size = batch_size
        return

    def __len__(self):  # returns number of batches
        return math.ceil((self.i2 - self.i1) / self.batch_size)

    def __getitem__(self, idx):  # returns one batch
        index = np.arange(self.i1 + idx * self.batch_size, min(self.i2, self.i1 + (idx + 1) * self.batch_size))
        batch_x = x[index]
        batch_y = y[index]
        return batch_x, batch_y

In [ ]:
# neural network
BS  = 1024 * 8  # batch size
D1  = 512  # width
tr_idx = np.arange(x.shape[0]) < x.shape[0] // 2
va_idx = np.arange(x.shape[0]) >= x.shape[0]//2
with tf.device('/GPU:0'):
    i1 = Input(shape=(x.shape[1],), dtype='float32')
    d1 = Dense(D1, activation='selu')(i1)
    o  = Dense(y.shape[1], activation=None)(d1)

    # compile the model
    model = tf.keras.Model(inputs=i1, outputs=o)
    model.compile(tf.keras.optimizers.Adam(0.001), loss='MSE')
    print(model.summary())
    es = EarlyStopping(monitor='val_loss', min_delta=.0001, patience=5, verbose=0, mode='min', restore_best_weights=True)
    tr = data_gen(0, x.shape[0]//2, BS)
    va = data_gen(x.shape[0]//2, x.shape[0], BS)
    model.fit(x=tr, validation_data=va, epochs=5, callbacks=[es], verbose=2)

    # predict in batches to avoid OOM
    predt = np.zeros([xt.shape[0], y.shape[1]], dtype=np.float32)
    i1 = 0
    BS2 = 1024 * 128
    for i in range(10000):
        i2 = np.minimum(i1 + BS2, xt.shape[0])
        predt[i1:i2,:] = model.predict(xt[i1:i2,:], verbose=0)
        i1 = i2
        print(np.round(i2/predt.shape[0], 2)) # pct completion
        if i2 >= xt.shape[0]:
            break

In [ ]:
# submit
# override constant columns
for i in range(sy.shape[0]):
    if sy[i] < min_std * 1.1:
        predt[:,i] = 0

# undo y scaling
predt = predt * sy.reshape(1,-1) + my.reshape(1,-1)

ss = pd.read_csv(os.path.join(CFG.data_path, "sample_submission.csv"))
ss.iloc[:,1:] *= predt
ss.to_csv("submission.csv", index=False)